In [22]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [23]:
with open('workplan.json') as f:
    json_data = json.load(f)

## The days, hours, and durations that a user may be available through the period.

In [24]:
AvailableDays =json_normalize(data=json_data['users'], record_path='availableDays',meta=['id'])
AvailableDays = pd.DataFrame(AvailableDays)
AvailableDays = AvailableDays[["id","to","from"]]
AvailableDays['to'] = pd.to_datetime(AvailableDays['to'], errors='coerce')
AvailableDays['from'] = pd.to_datetime(AvailableDays['from'], errors='coerce')
AvailableDays['to_date'] = AvailableDays['to'].dt.date
AvailableDays['to_time'] = AvailableDays['to'].dt.time
AvailableDays['from_date'] = AvailableDays['from'].dt.date
AvailableDays['from_time'] = AvailableDays['from'].dt.time
AvailableDays = AvailableDays.drop(['to' , 'from'] , axis = 1)
AvailableDays['to_date'] = pd.to_datetime(AvailableDays['to_date'], errors='coerce')
AvailableDays['from_date'] = pd.to_datetime(AvailableDays['from_date'], errors='coerce')
AvailableDays['from_day'] = AvailableDays['from_date'].dt.day_name()
AvailableDays['to_day'] = AvailableDays['to_date'].dt.day_name()
AvailableDays.head(5)

id    to_date   to_time  from_date from_time   from_day     to_day
0  1001 2020-03-23  12:00:00 2020-03-22  20:00:00     Sunday     Monday
1  1001 2020-03-24  12:00:00 2020-03-23  20:00:00     Monday    Tuesday
2  1001 2020-03-25  12:00:00 2020-03-24  20:00:00    Tuesday  Wednesday
3  1001 2020-03-26  12:00:00 2020-03-25  20:00:00  Wednesday   Thursday
4  1001 2020-03-27  12:00:00 2020-03-26  20:00:00   Thursday     Friday

## Minimum and maximum allocated hours through the period

In [25]:
allocated_hours  = json_normalize(data=json_data['users'])
allocated_hours = pd.DataFrame(allocated_hours)
allocated_hours = allocated_hours[["id","contract.paidHours.paidMinutesMin","contract.paidHours.paidMinutesMax"]]
cols = allocated_hours.columns.values
cols[1] = "paidMinutesMin"
cols[2] = "paidMinutesMax"
allocated_hours.columns = cols
allocated_hours["paidMinutesMin"] = allocated_hours["paidMinutesMin"]/60.0
allocated_hours["paidMinutesMax"] = allocated_hours["paidMinutesMax"]/60.0
cols = allocated_hours.columns.values
cols[1] = "allocatedHoursMin"
cols[2] = "allocatedHoursMax"
allocated_hours.columns = cols
allocated_hours.head(5)

id  allocatedHoursMin  allocatedHoursMax
0  1001               40.0               40.0
1  1002               40.0               40.0
2  1003               40.0               40.0
3  1004               40.0               40.0
4  1005               40.0               40.0

## Minimum time between shifts

In [26]:
i = 0
test = list()
while i< len(allocated_hours):
    test.append(json_data["users"][i]["contract"]["minimumTimeBetweenShiftsMins"])
    i = i+1
minimumTimeBetweenShiftsMins = pd.DataFrame()
minimumTimeBetweenShiftsMins["id"] = allocated_hours["id"]
minimumTimeBetweenShiftsMins["minimumTimeBetweenShiftsMins"] = test
minimumTimeBetweenShiftsMins.head(5)

id  minimumTimeBetweenShiftsMins
0  1001                           720
1  1002                           720
2  1003                           720
3  1004                           720
4  1005                           720

## Maximum consecutive (Shifts) working days

In [27]:
maxConsecutiveShifts =json_normalize(data=json_data['users'])
maxConsecutiveShifts = pd.DataFrame(maxConsecutiveShifts)
maxConsecutiveShifts = maxConsecutiveShifts[["id","contract.maxConsecutiveShifts"]]
cols = maxConsecutiveShifts.columns.values
cols[1] = "MaxConsecutiveShifts"
maxConsecutiveShifts.columns = cols
maxConsecutiveShifts.head(5)

id  MaxConsecutiveShifts
0  1001                     7
1  1002                     7
2  1003                     7
3  1004                     7
4  1005                     7

## The staff member must have the required skill to match the requirement

In [28]:
i = 0
test = list()
businessRoles = pd.DataFrame()
businessRoles["Business_id"] = 0
businessRoles["id"] = 0
while i< len(allocated_hours):
    test.append(json_data["users"][i]["businessRoles"])
    i = i+1
businessRoles["Business_id"] = test
businessRoles["id"] = allocated_hours.id
print(businessRoles.head(5))
businessRoles["Business_id"] = businessRoles["Business_id"].astype(str).str.replace(r"[^1-9 ]+", "").str.strip()
businessRoles[['id1','id2']]=businessRoles.Business_id.str.split(expand=True)
businessRoles=businessRoles.drop(['Business_id'],axis=1)
businessRoles=businessRoles.set_index('id').stack().reset_index(level=1,drop=True).to_frame('Buisness_ID').reset_index()
print(businessRoles.head(5))

                Business_id    id
0  [{'id': 23}, {'id': 24}]  1001
1  [{'id': 23}, {'id': 24}]  1002
2  [{'id': 23}, {'id': 24}]  1003
3  [{'id': 23}, {'id': 24}]  1004
4  [{'id': 23}, {'id': 24}]  1005
     id Buisness_ID
0  1001          23
1  1001          24
2  1002          23
3  1002          24
4  1003          23
